1. Import các thư viện cần thiết:

In [ ]:
import torch
import torch.nn as nn

seed = 1
torch.manual_seed(seed)

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from torch.utils.data import Dataset , DataLoader

2. Tải bộ dữ liệu: Các bạn tải bộ dữ liệu tại đây. Dưới đây là thông tin 4 hàng đầu tiên của
bảng dữ liệu:

3. Chuẩn bị dữ liệu:

(a) Đọc dữ liệu từ file .csv:

In [ ]:
dataset_filepath = 'dataset/weatherHistory.csv'
df = pd.read_csv(dataset_filepath)

Trong bài tập lần này, vì mô hình chỉ dự đoán nhiệt độ (Temperature (C)) nên ta sẽ loại
bỏ đi các cột không cần thiết trong DataFrame trước khi đưa vào tiền xử lý:

In [ ]:
univariate_df = df['Temperature (C)']
univariate_df.index = df['Formatted Date']

(b) Xây dựng hàm tạo cặp X, y:

In [ ]:
input_size = 6
label_size = 1
offset = 1

def slicing_window(df , df_start_idx , df_end_idx , input_size , label_size ,offset):
features = []
labels = []

window_size = input_size + offset

if df_end_idx == None:
df_end_idx = len(df) - window_size

for idx in range(df_start_idx , df_end_idx):
feature_end_idx = idx + input_size
label_start_idx = idx + window_size - label_size

feature = df[idx:feature_end_idx]
label = df[label_start_idx :(idx+window_size)]

features.append(feature)
labels.append(label)

features = np.expand_dims(np.array(features), -1)
labels = np.array(labels)

return features , labels

4. Chia bộ dữ liệu train, val, test: Sử dụng hàm slicing_window() đã định nghĩa ở trên, ta
tiến hành chia ba bộ dữ liệu train, val, test như sau:

In [ ]:
dataset_length = len(univariate_df)
train_size = 0.7
val_size = 0.2
train_end_idx = int(train_size * dataset_length)
val_end_idx = int(val_size * dataset_length) + train_end_idx

X_train , y_train = slicing_window(
univariate_df ,
df_start_idx =0,
df_end_idx=train_end_idx ,
input_size=input_size ,
label_size=label_size ,
offset=offset
)

X_val , y_val = slicing_window(
univariate_df ,
df_start_idx=train_end_idx ,
df_end_idx=val_end_idx ,
input_size=input_size ,
label_size=label_size ,
offset=offset
)

X_test , y_test = slicing_window(
univariate_df ,
df_start_idx=val_end_idx ,
df_end_idx=None ,
input_size=input_size ,
label_size=label_size ,
offset=offset
)

5. Xây dựng pytorch datasets:

In [ ]:
class WeatherForecast(Dataset):
    def __init__(
self ,
X, y,
transform=None
):
self.X = X
self.y = y
self.transform = transform

def __len__(self):
return len(self.X)

def __getitem__(self , idx):
X = self.X[idx]
y = self.y[idx]

if self.transform:
X = self.transform(X)

X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.float32)

return X, y

6. Khai báo dataloader:

In [ ]:
train_dataset = WeatherForecast(
X_train , y_train
)
val_dataset = WeatherForecast(
X_val , y_val
)
test_dataset = WeatherForecast(
X_test , y_test
)

train_batch_size = 128
test_batch_size = 8

train_loader = DataLoader(
train_dataset ,
batch_size=train_batch_size ,
shuffle=True
)
val_loader = DataLoader(
val_dataset ,
batch_size=test_batch_size ,
shuffle=False
)
test_loader = DataLoader(
test_dataset ,
batch_size=test_batch_size ,
shuffle=False
)

7. Xây dựng mô hình: 

In [ ]:
class WeatherForecastor(nn.Module):
def __init__(
self , embedding_dim , hidden_size ,
n_layers , dropout_prob
):
super(WeatherForecastor , self).__init__ ()
self.rnn = nn.RNN(embedding_dim , hidden_size , n_layers , batch_first=True)
self.norm = nn.LayerNorm(hidden_size)

self.dropout = nn.Dropout(dropout_prob)
self.fc = nn.Linear(hidden_size , 1)

def forward(self , x):
x, hn = self.rnn(x)
x = x[:, -1, :]
x = self.norm(x)
x = self.dropout(x)
x = self.fc(x)

return x

Sử dụng class đã định nghĩa ở trên, ta khai báo mô hình WeatherForecastor như sau:

In [ ]:
embedding_dim = 1
hidden_size = 8
n_layers = 3
dropout_prob = 0.2
device = 'cuda' if torch.cuda.is_available () else 'cpu'

model = WeatherForecastor(
embedding_dim=embedding_dim ,
hidden_size=hidden_size ,
n_layers=n_layers ,
dropout_prob=dropout_prob
).to(device)

8. Cài đặt hàm loss và optimizer: Vì bài toán dự đoán nhiệt độ thời tiết này dưới dạng là
bài Regression, ta sẽ sử dụng hàm loss là MSE và optimizer là Adam.

In [ ]:
lr = 1e-3
epochs = 50

criterion = nn.MSELoss ()
optimizer = torch.optim.Adam(
model.parameters (),
lr=lr
)

9. Thực hiện huấn luyện mô hình:

In [ ]:
train_losses , val_losses = fit(
model ,
train_loader ,
val_loader ,
criterion ,
optimizer ,
device ,
epochs
)

10. Đánh giá mô hình:

In [ ]:
val_loss = evaluate(
model ,
val_loader ,
criterion ,
device
)
test_loss = evaluate(
model ,
test_loader ,
criterion ,
device
)

print('Evaluation on val/test dataset ')
print('Val loss: ', val_loss)
print('Test loss: ', test_loss)

Bên cạnh đó, đối với bài toán time series, ta cũng có thể plot dự đoán của mô hình so với
dữ liệu thực tế tromg một khoảng thời gian như sau:

In [ ]:
def plot_difference(y, pred):
plt.figure(figsize =(20, 6))
times = range(len(y))
y_to_plot = y.flatten ()
pred_to_plot = pred.flatten ()

plt.plot(times , y_to_plot , color='steelblue ', marker='o', label='Truevalue')
plt.plot(times , pred_to_plot , color='orangered ', marker='X', label='Prediction ')

plt.title('Temperature in every hours')
plt.xlabel('Hour')
plt.ylabel('Temperature (C)')
plt.legend ()
plt.show()

inputs = torch.tensor(X_test [:100] , dtype=torch.float32).to(device)
model.eval()
with torch.no_grad ():
outputs = model(inputs).detach ().cpu().numpy ()
plot_difference(y_test [:100] , outputs)

Triển khai mô hình: Dựa trên mô hình đã huấn luyện và thư viện streamlit để triển khai ứng
dụng:<br>
– Link Streamlit<br>
– Github<br>
– Giao Diện<br>